In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime as dt
import matplotlib
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Training Data Paths
burn = pd.read_csv("burntrain.csv")
burn2 = pd.read_csv("burntest.csv")
burn.head()

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,fffe32003000360033003200,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,fffe3700360033003500,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
2,fffe31003300320037003900,2008-03-10,Female,Product,Yes,2.0,NaN,5.8,0.49
3,fffe32003400380032003900,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,fffe31003900340031003600,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52


In [3]:
burn2.head()

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score
0,fffe31003300390039003000,2008-12-10,Female,Service,No,2.0,5.0,7.7
1,fffe31003300310037003800,2008-08-14,Female,Product,Yes,1.0,2.0,5.2
2,fffe33003400380035003900,2008-11-13,Male,Product,Yes,1.0,3.0,5.9
3,fffe3100370039003200,2008-02-07,Female,Service,No,3.0,6.0,4.6
4,fffe32003600390036003700,2008-07-17,Female,Product,No,2.0,5.0,6.4


In [4]:
del burn['Employee ID']
del burn2['Employee ID']

In [5]:
burn = burn.replace(np.nan, 0)
burn2 = burn2.replace(np.nan, 0)
burn.head(5)

,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
2,2008-03-10,Female,Product,Yes,2.0,0.0,5.8,0.49
3,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52


In [6]:
burn['Date of Joining'] =  pd.to_datetime(burn["Date of Joining"])
burn2['Date of Joining'] =  pd.to_datetime(burn2["Date of Joining"])
burn.head()

,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
2,2008-03-10,Female,Product,Yes,2.0,0.0,5.8,0.49
3,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52


In [7]:
burn2.head()

,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score
0,2008-12-10,Female,Service,No,2.0,5.0,7.7
1,2008-08-14,Female,Product,Yes,1.0,2.0,5.2
2,2008-11-13,Male,Product,Yes,1.0,3.0,5.9
3,2008-02-07,Female,Service,No,3.0,6.0,4.6
4,2008-07-17,Female,Product,No,2.0,5.0,6.4


In [8]:
burn['Date of Joining'] = pd.DatetimeIndex(burn['Date of Joining']).month
burn.rename(columns = {'Date of Joining' : 'Month Joined'}, inplace = True)
burn2['Date of Joining'] = pd.DatetimeIndex(burn2['Date of Joining']).month
burn2.rename(columns = {'Date of Joining' : 'Month Joined'}, inplace = True)
burn.head(10)

,Month Joined,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,9,Female,Service,No,2.0,3.0,3.8,0.16
1,11,Male,Service,Yes,1.0,2.0,5.0,0.36
2,3,Female,Product,Yes,2.0,0.0,5.8,0.49
3,11,Male,Service,Yes,1.0,1.0,2.6,0.20
4,7,Female,Service,No,3.0,7.0,6.9,0.52
5,11,Male,Product,Yes,2.0,4.0,3.6,0.29
6,1,Female,Service,No,3.0,6.0,7.9,0.62
7,10,Female,Service,Yes,2.0,4.0,4.4,0.33
8,12,Female,Service,No,3.0,6.0,0.0,0.56
9,3,Female,Product,No,3.0,6.0,0.0,0.67


In [9]:
burn_df_clean = {"Gender": {"Male": 0, "Female": 1},
                  "Company Type": {"Service" : 0, "Product" : 1},
                    "WFH Setup Available": {"No" : 0, "Yes": 1}}
burn_new_train = burn.replace(burn_df_clean)
burn_new_test = burn2.replace(burn_df_clean)
burn_new_train.head()

,Month Joined,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,9,1,0,0,2.0,3.0,3.8,0.16
1,11,0,0,1,1.0,2.0,5.0,0.36
2,3,1,1,1,2.0,0.0,5.8,0.49
3,11,0,0,1,1.0,1.0,2.6,0.20
4,7,1,0,0,3.0,7.0,6.9,0.52


In [10]:
burn_new_test.head()

,Month Joined,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score
0,12,1,0,0,2.0,5.0,7.7
1,8,1,1,1,1.0,2.0,5.2
2,11,0,1,1,1.0,3.0,5.9
3,2,1,0,0,3.0,6.0,4.6
4,7,1,1,0,2.0,5.0,6.4


In [11]:
features = pd.get_dummies(burn_new_train)
features.head(5)

,Month Joined,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,9,1,0,0,2.0,3.0,3.8,0.16
1,11,0,0,1,1.0,2.0,5.0,0.36
2,3,1,1,1,2.0,0.0,5.8,0.49
3,11,0,0,1,1.0,1.0,2.6,0.20
4,7,1,0,0,3.0,7.0,6.9,0.52


In [12]:
features_test = pd.get_dummies(burn_new_test)
features_test.head()

,Month Joined,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score
0,12,1,0,0,2.0,5.0,7.7
1,8,1,1,1,1.0,2.0,5.2
2,11,0,1,1,1.0,3.0,5.9
3,2,1,0,0,3.0,6.0,4.6
4,7,1,1,0,2.0,5.0,6.4


In [13]:
# Labels are the values we want to predict
labels = np.array(features['Burn Rate'])# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop('Burn Rate', axis = 1)# Saving feature names for later use
feature_list = list(features.columns)# Convert to numpy array
features = np.array(features)

In [14]:
# The baseline predictions are the historical averages
baseline_preds = test_features[:, feature_list.index('Burn Rate')]# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

NameError: name 'test_features' is not defined